In [117]:
from ftplib import FTP
import os
import tempfile
import dicom2nifti

from typing import Callable
import importlib.util

import nibabel as nib
import numpy as np

import argparse

import pytest

In [125]:
conn = FTP('kel.osll.ru', 'storage', 'password')

In [126]:

conn.nlst('medicine/computer-tomography')

['medicine/computer-tomography/CQ500',
 'medicine/computer-tomography/RAS-hospital']

In [127]:
tmp_folder='./tmp'
try_convert_to_nii=True
remove_dicom_after_conversion=True
entrypoint='medicine/computer-tomography'

In [128]:
def traverse_ftp(conn: FTP, p: str):
    try:
        size = conn.size(p)
    except:
        size = 0

    if size == 0:
        files = conn.nlst(p)
        for file in files:
            for t in traverse_ftp(conn, file):
                if t == True:
                    yield p
                    return
                else:
                    yield t

    else:
        if p.endswith('.dcm'):
            yield True
        elif p.endswith('.nii'):
            yield p
            
def create_dirs(conn: FTP, path):
    root=''
    path = str.split(path, '/')
    print(path)
    for part in path:
        root = os.path.join(root, part)
        print(root)
        try:
            conn.mkd(root)
            print('ok')
        except:
            print('fail')
            pass

def add_to_nii_result(folder: str):
    nii_file_list.write(os.path.join(os.path.abspath(folder), 'result.nii') + '\n')
    nii_file_list.flush()

os.makedirs(os.path.join(tmp_folder, 'converted'), exist_ok=True)
with open(os.path.join(tmp_folder, 'converted', 'list.txt'), "w") as nii_file_list:
    for folder in traverse_ftp(conn, entrypoint):
        local_folder = os.path.join(tmp_folder, folder)

        if not os.path.exists(local_folder):
            print(f'loading {folder}')

            if folder.endswith('.nii'):
                nii_folder = os.path.join(local_folder, 'nii')
                os.makedirs(nii_folder)
                with open(os.path.join(nii_folder, 'result.nii'), 'wb') as f:
                    conn.retrbinary(f'RETR {folder}', f.write)
                print()
                print("!!!!!!!!!!FOUND NII!!!!!!!!!!")
                print(nii_folder)
                print()
                add_to_nii_result(nii_folder)
                continue

            os.makedirs(local_folder)
            for file in conn.nlst(folder):
                if file.endswith('.dcm'):
                    name = os.path.basename(file)
                    with open(os.path.join(tmp_folder, file), 'wb') as f:
                        conn.retrbinary(f'RETR {file}', f.write)

        if try_convert_to_nii:
            nii_folder = os.path.join(local_folder, 'nii')
            os.makedirs(nii_folder, exist_ok=True)
            try:
                print('converting to nii')
                filename = os.path.join(nii_folder, 'result.nii')
                if not os.path.exists(filename):
                    dicom2nifti.dicom_series_to_nifti(local_folder, filename)
                if remove_dicom_after_conversion:
                    for file in os.listdir(local_folder):
                        if file.endswith('.dcm'):
                            os.remove(os.path.join(local_folder, file))
                print()
                print("!!!!!!!!!!DONE!!!!!!!!!!")
                print(nii_folder)
                print()

                print()
                print('create dir')
                result_dir = os.path.join('tmp', folder, 'nii')
                create_dirs(conn, result_dir)
                print('done')
                print("sending to ftp server")
                filename = os.path.join(result_dir, 'result.nii')
                local_filename = os.path.join('./tmp', folder, 'nii', 'result.nii')

                if len(conn.nlst(result_dir))==0:
                    with open(local_filename, 'rb') as file:
                        print(filename)
                        try:
                            conn.storbinary(f'STOR {filename}', file)
                            print("!!!!!!!!!!DONE!!!!!!!!!!")
                        except:
                            print('fail to upload')
                            pass
                else:
                    print('skip')
                print()

                add_to_nii_result(nii_folder)
            except:
                os.rmdir(nii_folder)
                if remove_dicom_after_conversion:
                    for file in os.listdir(local_folder):
                        if file.endswith('.dcm'):
                            os.remove(os.path.join(local_folder, file))
                print('nope')

converting to nii
nope
converting to nii
nope
converting to nii
nope
converting to nii
nope
converting to nii
nope
converting to nii
nope
converting to nii
nope
converting to nii

!!!!!!!!!!DONE!!!!!!!!!!
./tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii


create dir
['tmp', 'medicine', 'computer-tomography', 'CQ500', 'CQ500CT100 CQ500CT100', 'Unknown Study', 'CT Plain 3mm', 'nii']
tmp
fail
tmp/medicine
fail
tmp/medicine/computer-tomography
fail
tmp/medicine/computer-tomography/CQ500
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii
fail
done
sending to ftp server

converting to nii

!!!!!!!!!!DONE!!!!!!!!!!
./tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ

Slice increment not consistent through all slices
---------------------------------------------------------
[ -94.    -116.     -66.632] [ 0.     0.    -3.744]
[ -94.   -116.    -64.76] [ 0.     0.    -1.872]
Instance Number: 21
---------------------------------------------------------


converting to nii
nope
loading medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study/CT 55mm Plain
converting to nii

!!!!!!!!!!DONE!!!!!!!!!!
./tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study/CT 55mm Plain/nii


create dir
['tmp', 'medicine', 'computer-tomography', 'CQ500', 'CQ500CT107 CQ500CT107', 'Unknown Study', 'CT 55mm Plain', 'nii']
tmp
fail
tmp/medicine
fail
tmp/medicine/computer-tomography
fail
tmp/medicine/computer-tomography/CQ500
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107
ok
tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study
ok
tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study/CT 55mm Plain
ok
tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study/CT 55mm Plain/nii
ok
done
sending to ftp server
tmp/medicine/computer-tomography/CQ500/CQ500CT107 CQ500CT107/Unknown Study/CT 55mm Plain/nii/result.nii
!!!!!!!!!!DONE!!!!!!!!!!

loading medicine/c

TimeoutError: [Errno 110] Connection timed out

In [90]:
def create_dirs(conn: FTP, path):
    root=''
    path = str.split(path, '/')
    print(path)
    for part in path:
        root = os.path.join(root, part)
        print(root)
        try:
            conn.mkd(root)
            print('ok')
        except:
            print('fail')
            pass



folder = 'medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm'

result_dir = os.path.join('tmp', folder, 'nii')
print(result_dir)
create_dirs(conn, result_dir)
filename = os.path.join(result_dir, 'result.nii')
local_filename = os.path.join('./tmp', folder, 'nii', 'result.nii')


with open(local_filename, 'rb') as file:
    print(filename)
    conn.storbinary(f'STOR {filename}', file)

tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii
['tmp', 'medicine', 'computer-tomography', 'CQ500', 'CQ500CT100 CQ500CT100', 'Unknown Study', 'CT Plain 3mm', 'nii']
tmp
ok
tmp/medicine
fail
tmp/medicine/computer-tomography
fail
tmp/medicine/computer-tomography/CQ500
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii
fail
tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii/result.nii


error_perm: 550 Create directory operation failed.

In [101]:
conn.nlst('tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii')

['tmp/medicine/computer-tomography/CQ500/CQ500CT100 CQ500CT100/Unknown Study/CT Plain 3mm/nii/result.nii']

In [98]:
conn.rmd('tmp/')

error_perm: 550 Remove directory operation failed.

In [123]:
conn.nlst('tmp/medicine/computer-tomography/CQ500/CQ500CT102 CQ500CT102/Unknown Study/CT Plain/nii')

[]